# Integrate Instant Recommendations With Your Existing Python SDK Model Workflows


---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-2/sagemaker-inference-recommender|python-sdk|instant-recommendations-with-python-sdk.ipynb)

---


* SDK https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model.deploy
* boto3 https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#client

In this notebook we will demonstrate the following Python SDK workflow:
1. Download the Model & Payload
2. Create the Python SDK model
3. Retrieve the Deployment Recommendation ID
4. (Optional) Display the recommendations
5. Deploy using a Deployment Recommendation ID
6. Perform inference off of the deployed model
7. Clean up

When demonstrating this Python SDK workflow, we will be using an SKLearn model.

**This sample is provided for demonstration purposes, make sure to conduct appropriate testing if deriving this code for your own use-cases!**

## Step 0: Make sure that you're using the latest Python SDK and Boto3 (Feel free to skip if already on latest versions)

In [ ]:
!pip install --upgrade pip botocore boto3 sagemaker

### Set up Client and Session

In [ ]:
import sagemaker
import boto3
import time

region = boto3.Session().region_name
role = sagemaker.get_execution_role()
sm_client = boto3.client("sagemaker", region_name=region)
sagemaker_session = sagemaker.Session()

## Step 1: Download the Model & Payload

In this example, we are using a pre-trained scikit-learn model, trained on the California Housing dataset, present in Scikit-Learn: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html. The California Housing dataset was originally published in:

> Pace, R. Kelley, and Ronald Barry. "Sparse spatial auto-regressions." Statistics & Probability Letters 33.3 (1997): 291-297.

### Download the Model

In [ ]:
import os

export_dir = "./model/"

if not os.path.exists(export_dir):
    os.makedirs(export_dir)
    print("Directory ", export_dir, " Created ")
else:
    print("Directory ", export_dir, " already exists")

model_archive_name = "sk-model.tar.gz"

In [ ]:
s3 = boto3.client("s3")
s3.download_file(
    f"sagemaker-example-files-prod-{region}",
    "models/california-housing/model.joblib",
    export_dir + "model.joblib",
)

### Tar the model and code

In [ ]:
!tar -cvpzf {model_archive_name} -C ./model "model.joblib" -C ../code "inference.py"

### Download the payload 

In [ ]:
payload_location = "./sample-payload/"

if not os.path.exists(payload_location):
    os.makedirs(payload_location)
    print("Directory ", payload_location, " Created ")
else:
    print("Directory ", payload_location, " already exists")

payload_archive_name = "sk_payload.tar.gz"

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
import pandas as pd

data = fetch_california_housing()

X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.25, random_state=42
)

# we don't train a model, so we will need only the testing data
testX = pd.DataFrame(X_test, columns=data.feature_names)
# Save testing data to CSV
testX[data.feature_names].head(10).to_csv(
    os.path.join(payload_location, "test_data.csv"), header=False, index=False
)

### Tar the payload

In [ ]:
!cd ./sample-payload/ && tar czvf ../{payload_archive_name} *

### Upload Your model and payload to S3

We will be uploading the pretrained model and corresponding test set as `sk-model.tar.gz` and as `sk_payload.tar.gz` to S3.

In [ ]:
prefix = "sagemaker/scikit-learn-inference-recommender"

model_url = sagemaker_session.upload_data(model_archive_name, key_prefix=prefix)
sample_payload_url = sagemaker_session.upload_data(payload_archive_name, key_prefix=prefix)

print("model uploaded to: {}".format(model_url))
print("sample payload uploaded to: {}".format(sample_payload_url))

## Step 2: Create the Python SDK model

In this example we will be using a Python SDK `SKLearnModel` model

In [ ]:
from sagemaker.model import Model
from sagemaker import image_uris

model = Model(
    model_data=model_url,
    role=role,
    image_uri=image_uris.retrieve(
        framework="sklearn", region=region, version="1.0-1", image_scope="inference"
    ),
    entry_point="./code/inference.py",
    sagemaker_session=sagemaker_session,
)

model.create()

## Step 3: Retrieve the Deployment Recommendation ID

In [ ]:
from datetime import datetime, timedelta

model_name = model.name

completed = timed_out = False

start = datetime.today()
max_duration = start + timedelta(minutes=1)

while not completed and not timed_out:
    describe_model_response = sm_client.describe_model(ModelName=model_name)
    deployment_recommendation = describe_model_response.get("DeploymentRecommendation")

    completed = (
        deployment_recommendation is not None
        and "COMPLETED" == deployment_recommendation.get("RecommendationStatus")
    )

    time.sleep(0.5)
    timed_out = datetime.today() > max_duration

    print(datetime.today(), "DeploymentRecommendation:", deployment_recommendation)

## (Optional) Step 4: Display the recommendations

In [ ]:
pd.set_option("display.max_colwidth", None)
df = pd.DataFrame(
    deployment_recommendation.get("RealTimeInferenceRecommendations"),
    columns=["RecommendationId", "InstanceType", "Environment"],
)
display(df)

## Step 5: Deploy using a Deployment Recommendation ID

In [ ]:
import uuid
from sagemaker.sklearn.model import SKLearnPredictor

recommendation_id = deployment_recommendation.get("RealTimeInferenceRecommendations")[0].get(
    "RecommendationId"
)

endpoint_name = "notebook-test-" + str(uuid.uuid4())

model.predictor_cls = SKLearnPredictor

predictor = model.deploy(
    inference_recommendation_id=recommendation_id,
    initial_instance_count=1,
    endpoint_name=endpoint_name,
)

## Step 6: Produce Inference

In [ ]:
import pandas as pd

payload = pd.read_csv("./sample-payload/test_data.csv")

inference = predictor.predict(payload)

print(inference)

## Step 7: Cleanup

Cleanup the resources if needed

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-1/sagemaker-inference-recommender|python-sdk|instant-recommendations-with-python-sdk.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-2/sagemaker-inference-recommender|python-sdk|instant-recommendations-with-python-sdk.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-1/sagemaker-inference-recommender|python-sdk|instant-recommendations-with-python-sdk.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ca-central-1/sagemaker-inference-recommender|python-sdk|instant-recommendations-with-python-sdk.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/sa-east-1/sagemaker-inference-recommender|python-sdk|instant-recommendations-with-python-sdk.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-1/sagemaker-inference-recommender|python|instant-recommendations-with-python-sdk.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-2/sagemaker-inference-recommender|python-sdk|instant-recommendations-with-python-sdk.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-3/sagemaker-inference-recommender|python-sdk|instant-recommendations-with-python-sdk.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-central-1/sagemaker-inference-recommender|python-sdk|instant-recommendations-with-python-sdk.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-north-1/sagemaker-inference-recommender|python-sdk|instant-recommendations-with-python-sdk.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-1/sagemaker-inference-recommender|python-sdk|instant-recommendations-with-python-sdk.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-2/sagemaker-inference-recommender|python-sdk|instant-recommendations-with-python-sdk.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-1/sagemaker-inference-recommender|python-sdk|instant-recommendations-with-python-sdk.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-2/sagemaker-inference-recommender|python-sdk|instant-recommendations-with-python-sdk.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-south-1/sagemaker-inference-recommender|python-sdk|instant-recommendations-with-python-sdk.ipynb)
